In [4]:
import ezdxf
fname = "../dxffiles/japonicalibrecad.dxf"
drawing = ezdxf.readfile(fname)


# Flattening and decoding the drawing is done by chaining together generators
def flatentitylistgenerator(drawing):  # ( (insertentity or None, entity) )
    insertentitystack = [ None ]
    while insertentitystack:
        insertentity = insertentitystack.pop()
        entities = drawing.entities  if insertentity is None  else drawing.blocks[insertentity.dxf.name]
        for entity in entities:
            if entity.dxftype() == 'INSERT':
                #print("INSERT translate", e.dxf.insert, "rotate", e.dxf.rotation, "scale", e.dxf.xscale, e.dxf.yscale)
                insertentitystack.append(entity)
            elif entity.dxftype() != 'ACAD_PROXY_ENTITY':
                yield (insertentity, entity)
    
def colorentitylayerlistgenerator(drawing):  # ( (layername, colornum, entity) )
    for insertentity, entity in flatentitylistgenerator(drawing):
        layername = entity.dxf.layer
        if entity.dxf.color == 256:
            layer = drawing.layers.get(layername)
            colnum = layer.get_color()
        elif entity.dxf.color == 0:
            if insertentity is not None:
                layer = drawing.layers.get(insertentity.dxf.layer)
                colnum = layer.get_color()
            else:
                colnum = 1
        else:
            colnum = entity.dxf.color
        yield (layername, colnum, entity)


def layergroupedentitygenerator(drawing):
    # pull all entities into a single list to sort and group by layer+color
    layercolorentities = list(colorentitylayerlistgenerator(drawing))

    # sort by layer name, but in the order that they appear in the drawing.layers list
    layernameorder = dict((n, i)  for i, n in enumerate(layer.dxf.name  for layer in drawing.layers))
    layercolorentities.sort(key=lambda X:(layernameorder.get(X[0], -1), X[1]))

    # Slice out the sequences of entities that share the same layer and color
    i = 0
    while i < len(layercolorentities):
        layername, colnum = layercolorentities[i][:2]
        j = i + 1
        while j < len(layercolorentities) and (layername, colnum) == layercolorentities[j][:2]:
            j += 1
        entities = [ entity  for (layername, colnum, entity) in layercolorentities[i:j] ]
        yield (layername, colnum, entities)
        i = j
        
dxftypesimplemented = ["LINE", "CIRCLE", "ARC", "SPLINE", "LWPOLYLINE", "POLYLINE"]
def countimplementedentities(entities, printfunc):
    entitytypecount = { }
    for entity in entities:
        dxftype = entity.dxftype()
        entitytypecount[dxftype] = entitytypecount.get(dxftype, 0)+1
    entitytypeset = set(entitytypecount.keys())
    
    res = 0
    for dxftype in dxftypesimplemented:
        if dxftype in entitytypeset:
            res += entitytypecount[dxftype]
            printfunc("   %s%s%s\n" % (dxftype, " "*max(1, 11-len(dxftype)), entitytypecount[dxftype]))
            entitytypeset.remove(dxftype)
    if entitytypeset:
        for dxftype in entitytypeset:
            printfunc("   **%s%s%s (not implemented)\n" % (dxftype, " "*max(1, 9-len(dxftype)), entitytypecount[dxftype]))
    return res
        
def printfunc(*x):  # or FreeCAD.Console.PrintMessage()
    print(*x, end="")
    
for layername, colnum, entities in layergroupedentitygenerator(drawing):
    print(layername, colnum)
    res = countimplementedentities(entities, printfunc)


0 7
   **TEXT     59 (not implemented)
1 1
   POLYLINE   59
8 7
   POLYLINE   328
NESTINGINFO 256
   **TEXT     7 (not implemented)


In [5]:
for layername, colnum, entities in layergroupedentitygenerator(drawing):
    print(layername, colnum, len(entities))

0 7 59
1 1 59
8 7 328
NESTINGINFO 256 7


In [25]:
d = list(drawing.entities)
#d[0].dxf.name, d[1].dxf.name
for i in d:
    if i.dxftype() == "INSERT":
        print(i.dxf.name, list(i.virtual_entities()))


PANEL2\9 []
PANEL2\8 []
PANEL2\2 []
PANEL2\7 []
PANEL3\8 []
PANEL3\2 []
PANEL2\1 []
PANEL3\7 []
PANEL3\1 []
PANEL2\10 []
PANEL3\6 []
PANEL1\4 []
PANEL2\3 []
PANEL3\3 []
PANEL3\4 []
PANEL2\4 []
PANEL1\9 []
PANEL1\3 []
PANEL1\1 []
PANEL4\2 []
PANEL1\5 []
PANEL2\5 []
PANEL1\7 []
PANEL1B\9 []
PANEL1\2 []
T1/3 []
H1/2 []
PANEL1B\4 []
PANEL1B\3 []
PANEL1B\2 []
PANEL3\5 []
H2 []
PANEL1B\1 []
C2 []
PANEL2\6 []
PANEL1\8 []
PANEL1\6 []
PANEL4\4 []
H1/3 []
PANEL4\3 []
PANEL1B\7 []
T1/2 []
PANEL1B\8 []
PANEL1B\5 []
PANEL4\1 []
PANEL4\5 []
C1/1 []
PANEL1B\6 []
PANEL4\6 []
C3 []
C4 []
C1/3 []
T1/1 []
H1/1 []
C1/2 []
T2 []
H4 []
H3 []
T3 []


In [23]:
b = drawing.blocks.get(i.dxf.name)
#b.virtual_entities
i.virtual_entities()

[]

In [33]:
d[0].dxfattribs()

{'layer': '1',
 'name': 'AAMADXFSAMPLE1',
 'insert': Vector(0.0, 0.0, 0.0),
 'handle': '18',
 'owner': '2D3'}

In [27]:
help(d[0])

Help on Insert in module ezdxf.entities.insert object:

class Insert(ezdxf.entities.subentity.LinkedEntities)
 |  DXF INSERT entity
 |  
 |  Method resolution order:
 |      Insert
 |      ezdxf.entities.subentity.LinkedEntities
 |      ezdxf.entities.dxfgfx.DXFGraphic
 |      ezdxf.entities.dxfentity.DXFEntity
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add_attrib(self, tag: str, text: str, insert: 'Vertex' = (0, 0), dxfattribs: dict = None) -> 'Attrib'
 |      Attach an :class:`Attrib` entity to the block reference.
 |      
 |      Example for appending an attribute to an INSERT entity with none
 |      standard alignment::
 |      
 |          e.add_attrib('EXAMPLETAG', 'example text').set_pos(
 |              (3, 7), align='MIDDLE_CENTER'
 |          )
 |      
 |      Args:
 |          tag: tag name as string
 |          text: content text as string
 |          insert: insert location as tuple ``(x, y[, z])`` in :ref:`WCS`
 |          dxfattribs: additional DX

In [26]:
drawing.blocks["AAMADXFSAMPLE1"].attribs

AttributeError: 'BlockLayout' object has no attribute 'attribs'

In [5]:
import ezdxf
from ezdxf.addons.dxf2code import entities_to_code, block_to_code

doc = ezdxf.readfile(fname)
msp = doc.modelspace()
source = entities_to_code(msp)

# create source code for a block definition
block_source = block_to_code(doc.blocks["AAMADXFSAMPLE1"])


In [8]:
#source.merge(block_source)   # crashes


In [9]:

# merge source code objects

with open('source.py', mode='wt') as f:
    f.write(source.import_str())
    f.write('\n\n')
    f.write(source.code_str())
    f.write('\n\n')
    f.write(block_source.import_str())
    f.write('\n\n')
    f.write(block_source.code_str())
    f.write('\n\n')
    
    

In [12]:
entities[0].dxf.layer

'14'

In [20]:
list(drawing.layers)[3].dxf.__dict__

{'_entity': <class 'ezdxf.entities.layer.Layer'> LAYER(#5),
 'name': '4',
 'flags': 0,
 'color': 3,
 'linetype': 'CONTINUOUS',
 'handle': '5',
 'owner': '2DB',
 'material_handle': '2F5',
 'plotstyle_handle': '2ED'}